In [3]:
!pip install --upgrade --force-reinstall xgboost==1.7.6 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [xgboost]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
numba 0

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [2]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [5]:
data=pd.read_csv('StudentsPerformance.csv')
data=pd.DataFrame(data)
data

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [6]:
Y=data['math score']
X=data[['gender','test preparation course','reading score','writing score']]
TrainX,TestX,TrainY,TestY=train_test_split(X,Y,random_state=0)
TrainX

,gender,test preparation course,reading score,writing score
253,male,none,80,72
667,female,none,85,87
85,female,none,80,82
969,female,none,84,80
75,male,none,41,38
...,...,...,...,...
835,female,completed,64,74
192,female,none,64,66
629,female,completed,51,55
559,male,none,66,62


# ***Pipelines***

In [7]:
num_cols=TrainX.select_dtypes(exclude='object').columns
obj_cols=TrainX.select_dtypes(exclude=['float','int']).columns
print(num_cols)
print(obj_cols)

Index(['reading score', 'writing score'], dtype='object')
Index(['gender', 'test preparation course'], dtype='object')


In [8]:
num_Transformer=SimpleImputer(strategy='constant')
cat_Transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(transformers=[
    ('num',num_Transformer,num_cols),
    ('cat',cat_Transformer,obj_cols)
])
model=RandomForestRegressor(n_estimators=100,random_state=1)
pipelines=Pipeline([
    ('preprocessor',preprocessor),
    ('model',model)
])
pipelines.fit(TrainX,TrainY)
pred=pipelines.predict(TestX)
mean_absolute_error(TestY,pred)

5.073911438672439

# ***Cross Validation***

In [9]:
validation=-1 * cross_val_score(pipelines,X,Y,cv=5,scoring='neg_mean_absolute_error')
np.mean(validation)

np.float64(5.298664021284271)

# ***XG Boost***

In [10]:
data=pd.read_csv('melb_data.csv')
data=pd.DataFrame(data)
data
Y=data['Price']
X=data[['Rooms','Distance','Bathroom','Car','Landsize','BuildingArea','Lattitude','Longtitude']]
TrainX,TestX,TrainY,TestY=train_test_split(X,Y,random_state=0)
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [31]:
# Train_X=pd.get_dummies(TrainX,columns=['gender','test preparation course'])
# Test_X=pd.get_dummies(TestX,columns=['gender','test preparation course'])

model=XGBRegressor(n_estimators=1000,learning_rate=0.01)
model.fit(TrainX,TrainY,eval_set=[(TestX,TestY)],eval_metric='mae',early_stopping_rounds=5,verbose=False)
pred=model.predict(TestX)
mean_absolute_error(TestY,pred)

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


178384.80179952134

In [16]:
model=RandomForestRegressor()
model.fit(TrainX,TrainY)
pred=model.predict(TestX)
mean_absolute_error(TestY,pred)

174462.9087335718